In [1]:
import pyvisa
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation

%matplotlib qt

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime
import time

import os
import h5py

Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'Daffing'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

C:\Users\Demag\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\Daffing already exists


In [2]:
now = datetime.now()
path = adress +'\\'+ 'Daffing scan'
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\Daffing\Daffing scan already exists


# VNA INIT


In [3]:
def magtodb(mag): return 20*np.log10(mag)

def vna_setup_CW(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('TCPIP0::169.254.36.111::inst0::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna


def vna_on(): vna.write('OUTP ON')
def vna_off(): vna.write('OUTP OFF')
    
def vna_meas_points(): 
    return int(vna.query('SENS1:SWE:POIN?'))  # should be 201

def vna_sweep_time(): 
    return float(vna.query('SENS1:SWE:TIME?'))  # estimated sweep time for current settings of vna

def vna_set_cw_freq(freq): 
    vna.write('SENS1:FREQ:CW {}'.format(str(freq))) # set single frequency point for CW mode in Hz

def vna_set_power(power): 
    vna.write('SOUR1:POW {}'.format(str(power))) # set power
    
def vna_set_band(bandwidth): 
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set power
    
def vna_meas_cw():
    vna_on()
    vna.write("INIT1:IMM")                              # start measurement
    time.sleep(0.2+float(vna.query('SENS1:SWE:TIME?'))) # this part is important becouse the measurement nedds to finnish before readout process
    vna.write("CALC1:FORM MLOG")                        # format of data
    data_str = vna.query("CALC1:DATA? SDAT")            # readout
    
    data = np.array(data_str.rstrip().split(",")).astype("float64")
    s = data[0::2] + 1j * data[1::2]
    i = data[0::2]
    q = data[1::2]
    return abs(s), angle(s)

In [4]:
rm = pyvisa.ResourceManager()
vna = rm.open_resource('TCPIP0::169.254.36.111::inst0::INSTR') # GPIB connection
vna.write('INIT1:CONT OFF')
vna.write(':SENS1:SWE:TYPE CW')  

20

In [5]:
# vna = vna_setup_CW() # initializing or checking for connection

In [6]:
fig, ax = plt.subplots(1, 2, figsize=(15,10))
xs = []
ys = []
m = np.array([])
p = np.array([])

freq = 5.7816347e9-12.4e3

freq_min = 5.78163263e9
freq_max = 5.78163798e9
power = -7
bandwidth = 10000

# min_amp = 1.135
# max_amp = 2.26
# HIST_BINS_x = np.linspace(min_amp, max_amp, 1000)

# min_pha = -1.6
# max_pha = -1.1
# HIST_BINS_y = np.linspace(min_pha, max_pha, 1000)

vna_set_power(power)
vna_set_band(bandwidth)
vna.write('SENS1:AVER:STAT OFF')
# This function is called periodically from FuncAnimation
def animate(i, xs, ys):
    if i==0: vna.write('SENS1:SWE:POIN 3')
    else: vna.write('SENS1:SWE:POIN 1000')
    # GET DATA
    
    vna_set_cw_freq(freq)
    Mag, Pha = vna_meas_cw()
#     Mag = np.random.randn(10)
#     Pha = np.random.randn(10)
    
#     S21 = Mag*np.exp(Pha)
    
    # Save new data to file
    
    Mag_list = Mag.tolist()
    Pha_list = Pha.tolist()
    
    # Add x and y to lists
    xs.extend(Mag_list)
    ys.extend(Pha_list)
    dict_data=dict()
    dict_data['freq'] = freq
    dict_data['power'] = power
    dict_data['vna_bandwidth'] = bandwidth

    dict_data['mag_s'] = np.array(xs)
    dict_data['pha_s'] = np.array(ys)

    experiment_name = 'S1_Hist 5.7816347e9-12.4e3'
    file_name = path+'\\'+experiment_name+'_'+now.strftime(r'%H-%M-%S')+'.mat'
    savemat(file_name, dict_data)
    
    # Draw x and y lists
    HIST_BINS_x = np.linspace(min(xs), max(xs), 100)
    HIST_BINS_y = np.linspace(min(ys), max(ys), 100)
    
    ax[0].clear()
    ax[0].hist(xs, bins=HIST_BINS_x, fc="green", alpha=0.8)
    
    ax[1].clear()
    ax[1].hist(ys, bins=HIST_BINS_y, fc="blue", alpha=0.8)
    
    # Format plot
    plt.xticks(rotation=45, ha='right')
    plt.title('Resonance fluctuations distribution')
    ax[0].set_ylim(0, 30)
    ax[1].set_ylim(0, 30)
    ax[0].set_ylabel('Number of events')
    ax[0].set_xlabel('Magnitude')
    ax[1].set_ylabel('Number of events')
    ax[1].set_xlabel('Phase')
    ax[0].text(1.7, 25, str(i), bbox=dict(facecolor='red', alpha=0.5))

# Set up plot to call animate() function periodically
ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys), interval=1000)
plt.show()

In [7]:
# dict_data=dict()
# dict_data['freq'] = freq
# dict_data['power'] = power
# dict_data['vna_bandwidth'] = bandwidth

# dict_data['mag_s'] = np.array(xs)
# dict_data['pha_s'] = np.array(ys)

# experiment_name = 'S1_Hist daff_stable_freq_band100'
# file_name = path+'\\'+experiment_name+'_'+now.strftime(r'%H-%M-%S')+'.mat'
# savemat(file_name, dict_data)

In [8]:
# fig, ax = plt.subplots(1, 2, figsize=(15,10))
# HIST_BINS_x = np.linspace(min(xs), max(xs), 10000)
# HIST_BINS_y = np.linspace(min(ys), max(ys), 10000)

# ax[0].clear()
# ax[0].hist(xs, bins=HIST_BINS_x, fc="green", alpha=0.8)

# ax[1].clear()
# ax[1].hist(ys, bins=HIST_BINS_y, fc="blue", alpha=0.8)

# # Format plot
# plt.xticks(rotation=45, ha='right')
# plt.title('Resonance fluctuations distribution')
# # ax[0].set_ylim(0, 30)
# # ax[1].set_ylim(0, 30)
# ax[0].set_ylabel('Number of events')
# ax[0].set_xlabel('Magnitude')
# ax[1].set_ylabel('Number of events')
# ax[1].set_xlabel('Phase')
# # ax[0].text(1.7, 25, str(i), bbox=dict(facecolor='red', alpha=0.5))
